# Pylablibsm API Example:

This file provides an example of using the calibration automation state-machine API

Sam Condon \
Caltech \
06/14/2021

In [2]:
import sys
import numpy as np
from inspect import signature
sys.path.append(r'..\\pylab\\')
from statemachine import State, StateMachine

from pylablib.pylablibsm import SM

## Actions

The state machine on its own does not execute any useful code. It simply provides a framework through which 
instrument control code can plug in through what is termed an 'action.' **An action is a collection of two pieces of data:**

*1) Function pointer* \
*2) Identifier string*

When actions are added to the state-machine, we pass both of the above pieces of information to the **\<state_machine_object\>.add_action_to_state()** method. Actions must be assigned to an individual state-machine state, so we also pass in a string to identify which state the action should execute within. Overall the call signature for the add_action() method is as follows:

*\<state_machine_object\>.add_action_to_state(\<state identifier string\>, \<action identifier string\>, \<action function pointer\>)*

All actions must accept a single positional argument. In the example actions below, this argument has been called 'action_dict.' When an action is called within the state-machine, it is passed a dictionary through the action_dict argument. This dictionary takes the following form:

action_dict = {'state_machine': \<state machine instance object\>, 'params': \<action parameters\>}

The state machine instance object can be used to control the state machine within actions while the action parameters define any arguments that the action needs to operate.


In [3]:
def init_action(action_dict):
    print("This is the function that initializes communication interfaces with all instruments.")

def waiting_action(action_dict):    
    print("Here we are waiting for user input.")
    print("*Query instruments for lost connections*")

def thinking_action(action_dict):
    print("This is the thinking action that generates the control loop parameters based on user input")    

def moving_action(action_dict):
    print("This is the moving action that moves all instruments into position for a measurement.")
    action_dict['state_machine'].control_loop_next()
    
def measuring_action(action_dict):
    print("This is the measuring action that reads all raw data.")
    action_dict['state_machine'].control_loop_next()
    
def checking_action(action_dict):
    print("This is the checking action to perform initial validation of raw data.")
    action_dict['state_machine'].control_loop_next()
    
def compressing_action(action_dict):
    print("This is the compressing action that compresses raw data before storage on the server.")
    action_dict['state_machine'].control_loop_next()
    
def writing_action(action_dict):
    print("This is the writing action that writes the compressed data to the server.")
    action_dict['state_machine'].control_loop_next()
    
def resetting_action(action_dict):
    print("This is the resetting action that resets all instruments to idle states.")
    print("Control loop complete!")
    action_dict['state_machine'].control_loop_complete()
    

#### Add all actions to the state machine using the call signature indicated in the comments above:

In [4]:
#instantiate state machine
state_machine = SM()

# Add all desired actions to the state-machine###########################
state_machine.add_action_to_state('Initializing', 'init_action_0', init_action)
state_machine.add_action_to_state('Waiting', 'waiting_action_0', waiting_action)
state_machine.add_action_to_state('Thinking', 'thinking_action_0', thinking_action)
state_machine.add_action_to_state('Moving', 'moving_action_0', moving_action)
state_machine.add_action_to_state('Measuring', 'measuring_action_0', measuring_action)
state_machine.add_action_to_state('Checking', 'checking_action_0', checking_action)
state_machine.add_action_to_state('Compressing', 'compressing_action_0', compressing_action)
state_machine.add_action_to_state('Writing', 'writing_action_0', writing_action)
state_machine.add_action_to_state('Resetting', 'resetting_action_0', resetting_action)



#### Start the state-machine by sending it into the Initializing state:
Note that any actions added above to the Initializing state will be executed after the state machine is started

In [5]:
state_machine.start_machine()

Initializing
This is the function that initializes communication interfaces with all instruments.


In [6]:
state_machine.init_to_wait()

Waiting


True

Here we are waiting for user input.
*Query instruments for lost connections*


In [7]:
state_machine.wait_to_think()

Thinking


True

This is the thinking action that generates the control loop parameters based on user input


In [8]:
state_machine.start_control_loop()

Moving


True

This is the moving action that moves all instruments into position for a measurement.
Measuring
This is the measuring action that reads all raw data.
Checking
This is the checking action to perform initial validation of raw data.
Compressing
This is the compressing action that compresses raw data before storage on the server.
Writing
This is the writing action that writes the compressed data to the server.
Resetting
This is the resetting action that resets all instruments to idle states.
Control loop complete!
Waiting
Here we are waiting for user input.
*Query instruments for lost connections*
